# Vertical scaling of columnar analysis: Numba and C++ integration

(no-frills notebook)

<br><br><br><br><br>

## Motivation

There are two kinds of languages, and Python is one of the slow kind.

<img src="img/benchmark-games-2023.svg" width="100%">

But there are a lot of ways to connect Python with compiled languages, usually C++ (increasingly, Rust).

<img src="img/history-of-bindings-2.svg" width="100%">

## Connecting some C++ code into Python